# Model Training, a Decision Tree, Random Forest and XGBoost

### Importing Libraries

In [5]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.tree import plot_tree
from sklearn import tree
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
import tensorflow
pd.set_option('display.max_columns', None)

In [6]:
final_dataset = pd.read_csv("../data/finalDataset.csv")
final_dataset

,AGE_DIFF,ATP_POINTS_DIFF,ATP_RANK_DIFF,BEST_OF,DRAW_SIZE,ELO_DIFF,ELO_GRAD_LAST_100_DIFF,ELO_GRAD_LAST_10_DIFF,ELO_GRAD_LAST_200_DIFF,ELO_GRAD_LAST_25_DIFF,ELO_GRAD_LAST_3_DIFF,ELO_GRAD_LAST_50_DIFF,ELO_GRAD_LAST_5_DIFF,ELO_SURFACE_DIFF,H2H_DIFF,H2H_SURFACE_DIFF,HEIGHT_DIFF,N_GAMES_DIFF,P_1ST_IN_LAST_100_DIFF,P_1ST_IN_LAST_10_DIFF,P_1ST_IN_LAST_200_DIFF,P_1ST_IN_LAST_25_DIFF,P_1ST_IN_LAST_3_DIFF,P_1ST_IN_LAST_50_DIFF,P_1ST_IN_LAST_5_DIFF,P_1ST_WON_LAST_100_DIFF,P_1ST_WON_LAST_10_DIFF,P_1ST_WON_LAST_200_DIFF,P_1ST_WON_LAST_25_DIFF,P_1ST_WON_LAST_3_DIFF,P_1ST_WON_LAST_50_DIFF,P_1ST_WON_LAST_5_DIFF,P_2ND_WON_LAST_100_DIFF,P_2ND_WON_LAST_10_DIFF,P_2ND_WON_LAST_200_DIFF,P_2ND_WON_LAST_25_DIFF,P_2ND_WON_LAST_3_DIFF,P_2ND_WON_LAST_50_DIFF,P_2ND_WON_LAST_5_DIFF,P_ACE_LAST_100_DIFF,P_ACE_LAST_10_DIFF,P_ACE_LAST_200_DIFF,P_ACE_LAST_25_DIFF,P_ACE_LAST_3_DIFF,P_ACE_LAST_50_DIFF,P_ACE_LAST_5_DIFF,P_BP_SAVED_LAST_100_DIFF,P_BP_SAVED_LAST_10_DIFF,P_BP_SAVED_LAST_200_DIFF,P_BP_SAVED_LAST_25_DIFF,P_BP_SAVED_LAST_3_DIFF,P_BP_SAVED_LAST_50_DIFF,P_BP_SAVED_LAST_5_DIFF,P_DF_LAST_100_DIFF,P_DF_LAST_10_DIFF,P_DF_LAST_200_DIFF,P_DF_LAST_25_DIFF,P_DF_LAST_3_DIFF,P_DF_LAST_50_DIFF,P_DF_LAST_5_DIFF,WIN_LAST_100_DIFF,WIN_LAST_10_DIFF,WIN_LAST_200_DIFF,WIN_LAST_25_DIFF,WIN_LAST_3_DIFF,WIN_LAST_50_DIFF,WIN_LAST_5_DIFF,RESULT
0,-5.0,-1028.0,69.0,3,32,0.000000,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.000000,0,0,-5.0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0
1,10.3,-257.0,126.0,3,32,0.000000,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.000000,0,0,8.0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0
2,-3.7,-352.0,135.0,3,32,0.000000,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.000000,0,0,0.0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0
3,-2.0,-19.0,7.0,3,32,0.000000,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.000000,0,0,-7.0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0
4,-0.9,-734.0,162.0,3,32,0.000000,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.000000,0,0,-11.0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

### Data Preprocessing, Scaling and Splitting, and Train-Test Split

In [8]:
# Convert data to numpy (exclude the first 5k matches, since ELO hasn't been properly calculated yet)
data = final_dataset.to_numpy(dtype=object)[5000:,:]
np.random.shuffle(data)

# Split the data using an 85% split between training and testing
split = 0.85
value = int(round(split*len(data)))

data_train = data[:value,:]
data_test = data[value:,:]

print("Training Data: "+str(data_train.shape))
print("Testing Data: "+str(data_test.shape))

Training Data: (76819, 68)
Testing Data: (13556, 68)


### Define mappers for the labels 

In [9]:
# Define several mappers
mapper = np.vectorize(lambda x: "Player 2 Wins" if x == 0 else "Player 1 Wins")
reverse_mapper = np.vectorize(lambda x: 0 if x == "Player 2 Wins" else 1)

# Training data
x_train = data_train[:,:-1]
y_pred_train = mapper(data_train[:,-1:]).squeeze()

# Testing data
x_test = data_test[:,:-1]
y_pred_test = mapper(data_test[:,-1:]).squeeze()

## Decision Tree

In [10]:
# Instantiate a Decision Tree
decision_sklearn = DecisionTreeClassifier(max_depth=4)
decision_sklearn = decision_sklearn.fit(x_train, y_pred_train)

# Make predictions and test accuracy
predictions_train = decision_sklearn.predict(x_train)
predictions_test = decision_sklearn.predict(x_test)
print("Train Accuracy: "+str(accuracy_score(y_pred_train, predictions_train)))
print("Test Accuracy: "+str(accuracy_score(y_pred_test, predictions_test)))

Train Accuracy: 0.658131451854359
Test Accuracy: 0.6593390380643258


In [11]:
text_representation = tree.export_text(decision_sklearn, feature_names=final_dataset.columns[:-1])
print(text_representation)

|--- ELO_DIFF <= -2.11
|   |--- ELO_DIFF <= -158.68
|   |   |--- ELO_DIFF <= -283.59
|   |   |   |--- ELO_SURFACE_DIFF <= -219.62
|   |   |   |   |--- class: Player 2 Wins
|   |   |   |--- ELO_SURFACE_DIFF >  -219.62
|   |   |   |   |--- class: Player 2 Wins
|   |   |--- ELO_DIFF >  -283.59
|   |   |   |--- ELO_SURFACE_DIFF <= -101.88
|   |   |   |   |--- class: Player 2 Wins
|   |   |   |--- ELO_SURFACE_DIFF >  -101.88
|   |   |   |   |--- class: Player 2 Wins
|   |--- ELO_DIFF >  -158.68
|   |   |--- ELO_SURFACE_DIFF <= -73.53
|   |   |   |--- AGE_DIFF <= -0.25
|   |   |   |   |--- class: Player 2 Wins
|   |   |   |--- AGE_DIFF >  -0.25
|   |   |   |   |--- class: Player 2 Wins
|   |   |--- ELO_SURFACE_DIFF >  -73.53
|   |   |   |--- ATP_RANK_DIFF <= 12.50
|   |   |   |   |--- class: Player 1 Wins
|   |   |   |--- ATP_RANK_DIFF >  12.50
|   |   |   |   |--- class: Player 2 Wins
|--- ELO_DIFF >  -2.11
|   |--- ELO_DIFF <= 112.84
|   |   |--- ATP_POINTS_DIFF <= 110.50
|   |   |   |--- 

### only taking elo in consideration, not good

## Random Forest

In [12]:
# Instantiate a Random Forsest
forest_sklearn = RandomForestClassifier(n_estimators=500, max_depth=10, max_features="sqrt", bootstrap=True)
forest_sklearn = forest_sklearn.fit(x_train, y_pred_train)

# Make predictions and test accuracy
predictions_train = forest_sklearn.predict(x_train)
predictions_test = forest_sklearn.predict(x_test)
print("Train Accuracy: "+str(accuracy_score(y_pred_train, predictions_train)))
print("Test Accuracy: "+str(accuracy_score(y_pred_test, predictions_test)))

Train Accuracy: 0.7110871008474466
Test Accuracy: 0.6728385954558866


In [13]:
# Instantiate a Random Forsest
forest_sklearn2 = RandomForestClassifier(n_estimators=100, max_depth=7, min_samples_split=400, min_samples_leaf=250, max_features="sqrt", bootstrap=True)
forest_sklearn2 = forest_sklearn2.fit(x_train, y_pred_train)

# Make predictions and test accuracy
predictions_train = forest_sklearn2.predict(x_train)
predictions_test = forest_sklearn2.predict(x_test)
print("Train Accuracy: "+str(accuracy_score(y_pred_train, predictions_train)))
print("Test Accuracy: "+str(accuracy_score(y_pred_test, predictions_test)))

Train Accuracy: 0.6702508493992372
Test Accuracy: 0.6719533785777515


In [15]:
# This is going to take a long time, if you want to comment it out (I already did this myself and the results are in the models folder)
param_grid = {
    'n_estimators': [100, 300],
    'max_depth': [5, 10, 15],
    'min_samples_split': [10, 20],
    'min_samples_leaf': [5, 10],
    'max_features': ['sqrt', 'log2']
}

grid_search = GridSearchCV(
    estimator=RandomForestClassifier(),
    param_grid=param_grid,
    cv=5, 
    n_jobs=-1,
    verbose=4
)
grid_search.fit(x_train, y_pred_train)

Fitting 5 folds for each of 48 candidates, totalling 240 fits


KeyboardInterrupt: 

In [14]:
# Best parameters
print("Best Parameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)

# Instantiate a Random Forsest
best_forest_model = RandomForestClassifier(max_depth=15, max_features='log2', min_samples_leaf=5, min_samples_split=20, n_estimators=300)
best_forest_model = best_forest_model.fit(x_train, y_pred_train)

# Make predictions and test accuracy
predictions_train = best_forest_model.predict(x_train)
predictions_test = best_forest_model.predict(x_test)
print("Train Accuracy: "+str(accuracy_score(y_pred_train, predictions_train)))
print("Test Accuracy: "+str(accuracy_score(y_pred_test, predictions_test)))

NameError: name 'grid_search' is not defined

## XGBoost

In [16]:
# Instantiate an XGBoost Classifier
xgb_model = XGBClassifier(n_estimators=200, max_depth=10, learning_rate=0.1, subsample=0.8, colsample_bytree=0.7)

# Train the model
xgb_model.fit(x_train, reverse_mapper(y_pred_train))

# Make predictions
predictions_train = xgb_model.predict(x_train)
predictions_test = xgb_model.predict(x_test)

# Calculate accuracy
print("Train Accuracy: " + str(accuracy_score(reverse_mapper(y_pred_train), predictions_train)))
print("Test Accuracy: " + str(accuracy_score(reverse_mapper(y_pred_test), predictions_test)))

Train Accuracy: 0.9754357645894896
Test Accuracy: 0.6624372971377988


In [17]:
# Sorting by importance in descending order
sorted_mapped_results = sorted(
    list(zip(final_dataset.columns[:-1], list(xgb_model.feature_importances_))),
    key=lambda x: x[1], 
    reverse=True
)

# Extracting sorted labels and their importances
sorted_labels = [label for label, importance in sorted_mapped_results]
sorted_importances = [importance for _, importance in sorted_mapped_results]

# Displaying results
for label, importance in sorted_mapped_results:
    print(f"{label}: {importance:.4f}")


ELO_DIFF: 0.0700
ELO_SURFACE_DIFF: 0.0461
ATP_POINTS_DIFF: 0.0286
BEST_OF: 0.0206
ATP_RANK_DIFF: 0.0165
AGE_DIFF: 0.0161
ELO_GRAD_LAST_200_DIFF: 0.0148
WIN_LAST_200_DIFF: 0.0143
ELO_GRAD_LAST_100_DIFF: 0.0140
P_1ST_WON_LAST_25_DIFF: 0.0140
P_1ST_WON_LAST_3_DIFF: 0.0139
P_2ND_WON_LAST_50_DIFF: 0.0139
ELO_GRAD_LAST_50_DIFF: 0.0138
P_2ND_WON_LAST_10_DIFF: 0.0138
P_1ST_IN_LAST_3_DIFF: 0.0137
H2H_SURFACE_DIFF: 0.0137
P_1ST_WON_LAST_10_DIFF: 0.0137
P_DF_LAST_50_DIFF: 0.0136
P_1ST_IN_LAST_200_DIFF: 0.0136
P_1ST_WON_LAST_5_DIFF: 0.0136
H2H_DIFF: 0.0136
P_1ST_WON_LAST_50_DIFF: 0.0135
P_ACE_LAST_50_DIFF: 0.0134
N_GAMES_DIFF: 0.0134
P_ACE_LAST_5_DIFF: 0.0134
P_2ND_WON_LAST_200_DIFF: 0.0133
P_DF_LAST_5_DIFF: 0.0133
ELO_GRAD_LAST_3_DIFF: 0.0133
P_BP_SAVED_LAST_25_DIFF: 0.0133
P_ACE_LAST_200_DIFF: 0.0132
P_1ST_WON_LAST_200_DIFF: 0.0132
WIN_LAST_50_DIFF: 0.0131
P_1ST_IN_LAST_25_DIFF: 0.0131
P_DF_LAST_100_DIFF: 0.0131
P_ACE_LAST_25_DIFF: 0.0131
DRAW_SIZE: 0.0131
P_BP_SAVED_LAST_200_DIFF: 0.0131
P_2ND_

In [ ]:
# Instantiate an XGBoost Classifier
xgb_model = XGBClassifier(
    n_estimators=100, 
    max_depth=5, 
    learning_rate=0.05, 
    subsample=0.7, 
    colsample_bytree=0.6,
    reg_alpha=0.1,
    reg_lambda=1.0
)

# Train the model
xgb_model.fit(x_train, reverse_mapper(y_pred_train))

# Make predictions
predictions_train = xgb_model.predict(x_train)
predictions_test = xgb_model.predict(x_test)

# Calculate accuracy
print("Train Accuracy: " + str(accuracy_score(reverse_mapper(y_pred_train), predictions_train)))
print("Test Accuracy: " + str(accuracy_score(reverse_mapper(y_pred_test), predictions_test)))

In [ ]:
xgb_model.save_model("../models/xgb_model.json")

In [ ]:
# Define parameter grid with all specified parameters
param_grid = {
    'n_estimators': [100, 300],
    'max_depth': [5, 10],
    'learning_rate': [0.01, 0.05],
    'subsample': [0.7],
    'colsample_bytree': [0.6],
    'reg_alpha': [0.1, 0.5],
    'reg_lambda': [0.5, 1.0]
}

# Instantiate an XGBoost Classifier
xgb_model = XGBClassifier()

# Perform GridSearchCV
grid_search = GridSearchCV(
    estimator=xgb_model, 
    param_grid=param_grid, 
    scoring='accuracy', 
    cv=5, 
    verbose=3, 
    n_jobs=-1
)
grid_search.fit(x_train, reverse_mapper(y_pred_train))

In [ ]:
# Best parameters
print("Best Parameters:", grid_search.best_params_)

# Train the best model
best_xgb_model = grid_search.best_estimator_

# Make predictions
predictions_train = best_xgb_model.predict(x_train)
predictions_test = best_xgb_model.predict(x_test)

# Calculate accuracy
print("Train Accuracy:", accuracy_score(reverse_mapper(y_pred_train), predictions_train))
print("Test Accuracy:", accuracy_score(reverse_mapper(y_pred_test), predictions_test))

In [ ]:
best_xgb_model = grid_search.best_estimator_
best_xgb_model.save_model("../models/final_xgb_model.json")